These are our imports

In [ ]:
#imports 2021.2.576440691
import numpy as np
print("numpy imported")
import json
print("json imported")
import time
print("time imported")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
print("tensorflow imported") 
from keras import Sequential
print("keras imported")
import config
channelID,serverID,targetID = config.getScope()
print("config imported")
import pickle
print("pickle imported")
import random
print("random imported")
import string
print("string imported")
from scipy.sparse import csc_matrix
print("scipy imported")

Run the section below to process your data

In [ ]:
def exists(filename):
    try:
        f = open(filename, "rb")
        f.close()
        return True
    except:
        return False
if exists("message.list") == False:
    channelID,serverID,targetID = config.getScope()
    daTestDoc = open("TestDoc","r",encoding="utf-8")
    daMessageList = eval(daTestDoc.read())
    with open('message.list','wb') as messageList_file:
        pickle.dump(daMessageList,messageList_file)
        print(":flushed:")
print('not flushed')

In [ ]:
inputList,outputList = [],[]
with open("message.list",'rb') as daMessageList:
    channelID,serverID,targetID = config.getScope()
    daMessageList = pickle.load(daMessageList)
    for i in range(len(daMessageList)):
        if i != len(daMessageList)+1:
            message = daMessageList[i]
            if message["author"]["id"] == str(targetID):
                #if i != 0:
                if True:
                    previousMessage = daMessageList[i-1]
                    if previousMessage["author"]["id"] == str(targetID):
                        if len(outputList) != 0:    
                            outputList[len(outputList)-1] += "   " + message["content"]
                    else:
                        if type(message.get("referenced_message"))!=dict:
                            outputList.append(message["content"])
                            message = daMessageList[i+1]
                            inputList.append(message["content"])
                        else:
                            inputList.append(message["referenced_message"]["content"])
                            outputList.append(message["content"])

        myIndice = random.randint(0,len(inputList))

    # for message in range(len(inputList)):
    # 	inputList[message] = removePunctuation(removeCaps(inputList[message]))
    with open('input.list','wb') as inputList_file:
        pickle.dump(inputList,inputList_file)
    with open('output.list','wb') as outputList_file:
        pickle.dump(outputList,outputList_file)
    print("Complete! (1/2)")


with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)
        
        removePunctuation = lambda x: ''.join(char for char in x if not char in string.punctuation)
        removeCaps = lambda x: x.lower()

        for message in range(len(inputList)):
            inputList[message] = removePunctuation(removeCaps(inputList[message]))
            outputList[message] = removePunctuation(removeCaps(outputList[message]))
        #inputList = np.array(inputList)
        #outputList = np.array(outputList)
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)

        print("Complete! (2/2)")

In [ ]:
daShape = None


with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)


        tokenizer = Tokenizer()
        #print(type(inputList+outputList))
        tokenizer.fit_on_texts(inputList)
        #print(inputList[0:10])
        inputList = tokenizer.texts_to_sequences(inputList)
        outputList = tokenizer.texts_to_sequences(outputList)

        #print(inputList)
        
        inputList = pad_sequences(inputList)
        outputList = pad_sequences(outputList)


        print(inputList.shape)
        print(outputList.shape)
        if inputList.shape[1] > outputList.shape[1]:
            daMaxLen = inputList.shape[1]
        else:
            daMaxLen = outputList.shape[1]
        print(daMaxLen)

        inputList = pad_sequences(inputList, maxlen = daMaxLen)
        outputList = pad_sequences(outputList, maxlen = daMaxLen)

        daShape = outputList.shape

        

        inputList = np.expand_dims(inputList,axis=0)
        outputList = np.expand_dims(outputList,axis=0)

        inputList = inputList.astype(np.uint8)
        outputList = outputList.astype(np.uint8)
        print(inputList[0:10])
        print(inputList.shape)
        inputList,outputList = inputList[0],outputList[0]
        print(inputList.shape)
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)
        print("Complete!")
print("sus")

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop

In [ ]:
#model config
randomIndice = random.randint(0,len(inputList)-1)

dataSize = 317 #number of features

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Flatten
x = 32

visInput = Input(shape = (dataSize))
vocabSize = len(tokenizer.word_index) #gets the vocab size from tokenizer, duh
EmbeddingLayer = Embedding(input_dim=vocabSize,output_dim=x,input_length=daMaxLen)#vocabsize, number of output dimensions(try 8), inputlength(the padded length of the things)

EmbeddingOutput = EmbeddingLayer(visInput)

flattenedOut = Flatten()(EmbeddingOutput)
EncoderLSTM = LSTM(317,input_shape=(inputList.shape[1],x),return_sequences=True,return_state=True)#number of neurons, input shape arg1 is number of timesteps, arg2 is dimensions of words/embeddings
encoder_output,encoder_h,encoder_c = EncoderLSTM(EmbeddingOutput)
print(encoder_output.shape)
DecoderLSTM = LSTM(317,input_shape=(inputList.shape[1],x),return_sequences=False,return_state=False)
decoder_output = DecoderLSTM(encoder_output)

In [ ]:
# #Inverse tokenization
# reverse_word_map = tokenizer.word_index
# #print(reverse_word_map.keys())
# reverse_word_map = {v: k for k, v in reverse_word_map.items()}
# reverse_word_map[0] = ""

#Model

inputList = np.array(inputList)
outputList = np.array(outputList)
print(inputList.shape,outputList.shape)

batch_size,epochs=10,1

training_model = Model(inputs=visInput, outputs=decoder_output)#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')#Training
training_model.fit(x=inputList,y=outputList, batch_size = batch_size, epochs = epochs)
training_model.save('julytraining.h5')

Stats

In [ ]:
training_model.summary()

In [ ]:
testText = np.array(inputList[0])
print(testText)
print(type(testText))
training_model = keras.models.load_model("julytraining.h5",compile=True)
print(type(testText))
modeloutput = training_model.predict(testText)
print(modeloutput)

In [ ]:
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        inputWordCounter = 0
        inputCharCounter = 0
        outputWordCounter = 0
        outputCharCounter = 0
        for message in range(len(inputList)):
            for char in inputList[message]:
                if char == " ":
                    inputWordCounter += 1
                inputCharCounter += 1
        for message in range(len(inputList)):
            for char in outputList[message]:
                if char == " ":
                    outputWordCounter += 1
                outputCharCounter += 1
        print("Count of Input Words: " + str(inputWordCounter))
        print("Count of Input Characters: " + str(inputCharCounter))
        print("Count of Output Words: " + str(outputWordCounter))
        print("Count of Output Characters: " + str(outputCharCounter))
print("sus")

In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)